In [2]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from typing_extensions import TypedDict
from langchain_groq import ChatGroq
from langchain import hub
# from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder, PromptTemplate
from langchain.memory import ConversationBufferMemory
from langgraph.graph import StateGraph, MessagesState
from typing import Dict, List, Annotated, TypedDict, Literal, Union
from langchain_core.messages import FunctionMessage
from langgraph.graph import StateGraph, START, END
from IPython.display import Image, display
import json
from pydantic import BaseModel, Field
from typing import List, Dict, Any
from langchain.output_parsers import PydanticOutputParser
from langchain.agents import AgentExecutor, Tool, initialize_agent, AgentType

load_dotenv()

os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")

In [3]:
# Define the output structure for requirements
class Requirement(BaseModel):
    id: str = Field(description="Unique identifier for the requirement")
    title: str = Field(description="Short title for the requirement")
    description: str = Field(description="Detailed description of what the requirement entails")
    priority: str = Field(description="Priority level (High, Medium, Low)")
    category: str = Field(description="Category of requirement (Functional, Non-functional, UI/UX, etc.)")
    acceptance_criteria: List[str] = Field(description="List of criteria that must be met for this requirement to be considered complete")


In [4]:
class RequirementsList(BaseModel):
    requirements: List[Requirement] = Field(description="List of extracted requirements")

In [5]:
def extract_requirements(user_input: str) -> Dict[str, Any]:
    """
    Extract structured requirements from user input.
    This will identify explicit and implicit requirements from the text.
    
    Args:
        user_input (str): The input text containing requirement descriptions
        
    Returns:
        Dict[str, Any]: Extracted requirements with structure
    """
    return {"status": "Requirements extracted", "input": user_input}

In [6]:
def clarify_requirements(question: str) -> str:
    """
    Generate clarifying questions for ambiguous requirements.
    
    Args:
        question (str): The requirement or context that needs clarification
        
    Returns:
        str: Clarification questions for the requirement
    """
    return f"Clarification needed: {question}"

In [7]:
def prioritize_requirements(requirements_json: str) -> str:
    """
    Suggest prioritization for the extracted requirements.
    
    Args:
        requirements_json (str): JSON string of requirements to prioritize
        
    Returns:
        str: Prioritized requirements with justification
    """
    return f"Prioritized: {requirements_json}"

In [8]:
def validate_requirements(requirements_json: str) -> str:
    """
    Check requirements for consistency, completeness, and clarity.
    
    Args:
        requirements_json (str): JSON string of requirements to validate
        
    Returns:
        str: Validation results and suggestions
    """
    return f"Validated: {requirements_json}"

In [9]:
# Create the requirements agent using Groq
def create_requirements_agent(groq_api_key=None, model_name="llama3-8b-8192", temperature=0.2):
    # Set API key if provided
    if groq_api_key:
        os.environ["GROQ_API_KEY"] = groq_api_key
    
    # Initialize the Groq language model
    llm = ChatGroq(
        model_name=model_name,
        temperature=temperature
    )
    
    # Define the tools
    tools = [
        Tool(
            name="ExtractRequirements",
            func=extract_requirements,
            description="Extract structured requirements from user input. Use this to identify and structure requirements from text."
        ),
        Tool(
            name="ClarifyRequirements",
            func=clarify_requirements,
            description="Generate questions to clarify ambiguous requirements. Use this when requirements are unclear or incomplete."
        ),
        Tool(
            name="PrioritizeRequirements",
            func=prioritize_requirements,
            description="Suggest prioritization for requirements. Use this to rank requirements by importance and impact."
        ),
        Tool(
            name="ValidateRequirements",
            func=validate_requirements,
            description="Check requirements for consistency and completeness. Use this to verify requirements quality."
        )
    ]
    
    # Create memory
    memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
    
    # Create the agent executor using the zero-shot-react-description approach
    # This approach doesn't require explicit tool_names and tools in the prompt
    agent_executor = initialize_agent(
        agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
        tools=tools,
        llm=llm,
        verbose=True,
        max_iterations=5,
        memory=memory,
        handle_parsing_errors=True,
        early_stopping_method="generate",
    )
    
    return agent_executor

In [10]:
# Example usage
def process_requirements(user_input: str, groq_api_key=None) -> Dict[str, Any]:
    requirements_agent = create_requirements_agent(groq_api_key)
    result = requirements_agent.invoke({"input": user_input})
    return result

In [11]:
user_input = """ DreamBigin - Web Application Requirements
A social media platform for artists to collaborate and showcase their work.

1. User Management
1.1 Registration & Login
Users should be able to register and log in using OAuth authentication (Google, Twitter, Facebook, etc.).
Users should have the option to register with email and password as an alternative.
Implement two-factor authentication (2FA) for enhanced security.
Implement role-based access control for different types of users (Artists, Businesses, Admins).
1.2 User Profile
Users should be able to create and manage their profile with:
Profile Picture
Bio / Short Description
List of Skills (e.g., Acting, Guitar, Directing)
Social Media Links (Instagram, YouTube, TikTok, LinkedIn, Twitter, etc.)
Portfolio Section (Showcase previous work with images/videos)
1.3 Collaboration Requests
Users should be able to create collaboration projects specifying:
Title
Project Description
Required Skills (e.g., Cinematographer, Scriptwriter, Vocalist)
Project Type (Short Film, Music Band, Art Collaboration, etc.)
Collaboration Location (Remote / Onsite)
Deadline (if applicable)
Compensation (if any)
Interested artists can apply to collaborate or directly message the project owner.
2. Business Enrollment & Advertisements
2.1 Business Accounts
Businesses (e.g., Camera Rentals, Film Institutes) should be able to:
Register as a Business Account.
List services (e.g., Equipment Rental, Online/Offline Courses).
Promote offers and discounts on rentals or training programs.
Receive inquiries from artists through direct messaging.
2.2 Business Promotions
Business accounts should have an option to boost their services (paid promotion).
Integration with Google Ads / Facebook Ads API for external promotions.
Businesses should be able to post job openings (e.g., “Seeking Assistant Director for an upcoming project”).
3. Content Sharing & Engagement
3.1 Content Creation
Users should be able to write articles and blogs (like Medium).
Users can post external links (e.g., YouTube, Instagram, SoundCloud).
Users can upload images & video snippets to showcase their work.
3.2 Engagement Features
Like, Comment, and Share features for posts.
Follow / Unfollow feature to subscribe to other users’ content.
Trending Content Section based on user engagement.
Hashtags & Categories for easy discoverability.
Users should be able to embed multimedia (videos, audio, GIFs).
4. Messaging & Notifications
4.1 Direct Messaging
Real-time chat for users to discuss collaborations.
Businesses can communicate with artists about services.
Group Chats for multi-person collaboration discussions.
4.2 Notifications
In-app notifications for new collaboration requests, messages, likes, comments, etc.
Email & Push Notifications (configurable in settings).
Weekly summary emails about trending posts and collaboration opportunities.
5. Discoverability & Search
5.1 Search & Filters
Search for Users, Projects, Businesses, and Content.
Filters by Skills, Location, Availability, Experience Level.
AI-powered recommendations based on user interests and engagement.
5.2 Featured Artists & Businesses
A curated list of top-rated artists based on collaborations and community engagement.
Business Spotlights showcasing top rental providers and training institutes.
6. Monetization & Payments
6.1 Premium Features
Subscription-based premium profiles for enhanced visibility.
Paid promoted collaboration requests to reach more artists.
6.2 Payment Gateway
Integration with Stripe / PayPal for in-app payments.
Artists can donate to other artists or fund a project.
Businesses can accept payments for courses & rentals.
7. Admin Panel
User & Content Moderation (Report & Ban system for abusive content).
Collaboration Request Monitoring (Prevent spam/fraudulent requests).
Analytics Dashboard (Track engagement, active users, trending skills).
8. Technology Stack (Suggested)
Frontend: React / Angular with TypeScript & SCSS.
Backend: .NET Core (C#) or Node.js (Express) with REST APIs.
Database: PostgreSQL / MySQL for structured data.
Authentication: OAuth, Firebase Auth, or Identity Server.
Messaging & Notifications: WebSockets for real-time chat, Firebase for push notifications.
File Storage: AWS S3 / Azure Blob Storage for multimedia.
AI Integration: AI-powered recommendations for matching artists & projects.
Additional Enhancements
✅ AI-based Profile Recommendations – Suggests potential collaborations based on skills.
✅ NFT Art Marketplace (Future Scope) – Artists can sell digital artwork as NFTs.
✅ Live Streaming Events – Artists and businesses can host live Q&A sessions.
✅ Mobile App (Future Scope) – Extend functionality to iOS/Android apps. """

In [12]:
# process_requirements(user_input)
# Example of how to use with Groq
def requirements_agent_example_groq(groq_api_key=None):
    # Creating a standalone example
    req_agent = create_requirements_agent(groq_api_key)
    
    # Example user input
    user_input = """
    We need a user authentication system that allows users to register, login, and reset their 
    passwords. Users should be able to use social login options like Google and Facebook.
    The system needs to be secure and should use two-factor authentication for sensitive operations.
    The login page should be responsive and work well on mobile devices.
    """
    
    # Process the requirements
    result = req_agent.invoke({"input": user_input})
    print(result)
    
    return result


In [13]:
result = requirements_agent_example_groq()

C:\Users\Deepak\AppData\Local\Temp\ipykernel_12628\3763589553.py:38: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
C:\Users\Deepak\AppData\Local\Temp\ipykernel_12628\3763589553.py:42: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent_exec



> Entering new AgentExecutor chain...
Thought: The user input seems to be a high-level description of the requirements for a user authentication system. It mentions specific features such as registration, login, password reset, social login options, security, two-factor authentication, and responsiveness on mobile devices.

Action: ExtractRequirements
Action Input: "We need a user authentication system that allows users to register, login, and reset their passwords. Users should be able to use social login options like Google and Facebook. The system needs to be secure and should use two-factor authentication for sensitive operations. The login page should be responsive and work well on mobile devices."
Observation: {'status': 'Requirements extracted', 'input': 'We need a user authentication system that allows users to register, login, and reset their passwords. Users should be able to use social login options like Google and Facebook. The system needs to be secure and should use two

In [14]:
result

{'input': '\n    We need a user authentication system that allows users to register, login, and reset their \n    passwords. Users should be able to use social login options like Google and Facebook.\n    The system needs to be secure and should use two-factor authentication for sensitive operations.\n    The login page should be responsive and work well on mobile devices.\n    ',
 'chat_history': [HumanMessage(content='\n    We need a user authentication system that allows users to register, login, and reset their \n    passwords. Users should be able to use social login options like Google and Facebook.\n    The system needs to be secure and should use two-factor authentication for sensitive operations.\n    The login page should be responsive and work well on mobile devices.\n    ', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Here's the continuation:\n\nAction: PrioritizeRequirements\nAction Input: The extracted requirements from the user input", additional_kwa

In [15]:
# Example implementation of a LangGraph compatible node
class RequirementsNode:
    def __init__(self, groq_api_key=None, model_name="llama3-8b-8192"):
        self.agent = create_requirements_agent(groq_api_key, model_name)
    
    def __call__(self, state):
        # Extract input from state
        user_input = state.get("user_input", "")
        
        # Process the requirements
        result = self.agent.invoke({"input": user_input})
        
        # Update the state with processed requirements
        updated_state = state.copy()
        updated_state["requirements"] = result.get("output", "")
        updated_state["requirements_raw"] = result
        
        return updated_state